# Notebook for End to End testing in the MID ITF

- Uses digitizers for the 2 inputs to the correlator
- Uses 1 FSP (200 MHz)
- Receives and plots the visibilities to SDP

# Initialise

## Select K8S namespace

In [37]:
import kubernetes
import ipywidgets as widgets
from IPython.display import display, JSON

def show_textbox(box_value, box_desc):
    textbox_w = widgets.Text(
        value=box_value,
        description=box_desc,
        disabled=True,
        layout=widgets.Layout(width='60%')
    )
    display(textbox_w)
    
def show_errorbox(box_value, error_desc="Error"):
    errorbox_w = widgets.Textarea(
        value=box_value,
        placeholder='Error message',
        description=error_desc,
        disabled=False,
        layout=widgets.Layout(width='75%', height='200px')
    )
    display(errorbox_w)

ns_list = []
ns_list.append("SELECT")
kubernetes.config.load_kube_config()
k8s_client = kubernetes.client.CoreV1Api()
try:
    namespaces = k8s_client.list_namespace(_request_timeout=(1, 5))
    for namespace in namespaces.items:
        ns_name = namespace.metadata.name
        ns_list.append(ns_name)
except Exception:
    pass

ns_valid_w = widgets.Valid(
    value=len(ns_list) > 0,
    description="Namespaces",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(ns_valid_w)

if len(ns_list) > 0:
    ns_w = widgets.Dropdown(
        options=ns_list,
        description='SUT ns',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )
    display(ns_w)

hack_w = widgets.Checkbox(
    value=True,
    description="Apply hacks",
    disabled=False,
    indent=False
)
display(hack_w)

cbf_sim_w = widgets.Checkbox(
    value=True,
    description="CBF simulation mode",
    disabled=False,
    indent=False
)
display(cbf_sim_w)

tel_on_wait = widgets.IntSlider(
    value=180,
    min=0,
    max=300,
    step=1,
    description='Wait time',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(tel_on_wait)

Valid(value=True, description='Namespaces', layout=Layout(width='60%'), style=DescriptionStyle(description_wid…

Dropdown(description='SUT ns', layout=Layout(width='50%'), options=('SELECT', 'binderhub', 'calico-apiserver',…

Checkbox(value=True, description='Apply hacks', indent=False)

Checkbox(value=True, description='CBF simulation mode', indent=False)

IntSlider(value=180, continuous_update=False, description='Wait time', max=300)

## Set dish names

In [38]:
import os
from time import sleep

# SUT_NAMESPACE = "staging"
SUT_NAMESPACE = ns_w.value
if SUT_NAMESPACE == "SELECT":
    print("ERROR: namespace not set")
else:
    print("Setting up parameters\n")

    APPLY_HACKS = hack_w.value
    if cbf_sim_w.value:
        CBF_SIMULATION_MODE = 1
    else:
        CBF_SIMULATION_MODE = 0

    # Set manually
    CLUSTER_DOMAIN = "miditf.internal.skao.int"

    if SUT_NAMESPACE == "staging":
        SKA001_NAMESPACE = (
            f"staging-dish-lmc-ska001"  # ci-dish-lmc-ska001-at-2139-tmc-0-20-1-integration
        )
        SKA036_NAMESPACE = (
            f"staging-dish-lmc-ska036"  # ci-dish-lmc-ska036-at-2139-tmc-0-20-1-integration
        )
    else:
        SKA001_NAMESPACE = f"ci-dish-lmc-ska001-{SUT_NAMESPACE[15:]}"  # ci-dish-lmc-ska001-at-2139-tmc-0-20-1-integration
        SKA036_NAMESPACE = f"ci-dish-lmc-ska036-{SUT_NAMESPACE[15:]}"  # ci-dish-lmc-ska036-at-2139-tmc-0-20-1-integration


    show_textbox(SUT_NAMESPACE, 'SUT ns')
    show_textbox(SKA001_NAMESPACE, 'SKA001 ns')
    show_textbox(SKA036_NAMESPACE, 'SKA036 ns')

    TANGO_HOST = f"tango-databaseds.{SUT_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
    os.environ["TANGO_HOST"] = TANGO_HOST
    show_textbox(os.getenv("TANGO_HOST"), 'Tango host')

    apply_hacks_w = widgets.Valid(
        value=APPLY_HACKS,
        description="Apply hacks",
        layout=widgets.Layout(width='60%'),
        style = {'description_width': 'initial'}
    )
    display(apply_hacks_w)

    apply_sim_w = widgets.Valid(
        value=APPLY_HACKS,
        description="CBF simulation",
        layout=widgets.Layout(width='60%'),
        style = {'description_width': 'initial'}
    )
    display(apply_sim_w)

    RECEPTORS = ["SKA001", "SKA036"]

    # Config files set up
    DATA_DIR = "../../data"
    TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
    SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
    RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

    ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
    CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

    CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
    DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

    KAFKA_PORT = 9092
    KAFKA_SERVICE_NAME = "ska-sdp-kafka"
    KAFKA_ENDPOINT = f"{KAFKA_SERVICE_NAME}.{SUT_NAMESPACE}.svc.{CLUSTER_DOMAIN}:{KAFKA_PORT}"

    print("Links for Taranta and QA Display\n")
    print("SUT Links")
    print(f"https://k8s.{CLUSTER_DOMAIN}/{SUT_NAMESPACE}/signal/display/")
    print(f"https://k8s.{CLUSTER_DOMAIN}/{SUT_NAMESPACE}/taranta/devices")
    print("\n")
    print("Dish LMC links")
    print(f"https://k8s.{CLUSTER_DOMAIN}/{SKA001_NAMESPACE}/taranta/devices")
    print(f"https://k8s.{CLUSTER_DOMAIN}/{SKA036_NAMESPACE}/taranta/devices")


Setting up parameters



Text(value='ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks', description='SUT ns', disabled=True, layout…

Text(value='ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks', description='SKA001 ns', disabled=True,…

Text(value='ci-dish-lmc-ska036-at-2146-consolidate-jupyter-notebooks', description='SKA036 ns', disabled=True,…

Text(value='tango-databaseds.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int…

Valid(value=True, description='Apply hacks', layout=Layout(width='60%'), style=DescriptionStyle(description_wi…

Valid(value=True, description='CBF simulation', layout=Layout(width='60%'), style=DescriptionStyle(description…

Links for Taranta and QA Display

SUT Links
https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks/signal/display/
https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks/taranta/devices


Dish LMC links
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks/taranta/devices
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska036-at-2146-consolidate-jupyter-notebooks/taranta/devices


# Set Up and Configure

## Device Proxies 

In [39]:
import json

from tango import DeviceProxy
print("Setting up the device proxies")

proxy_w = widgets.IntProgress(
    value=0,
    min=0,
    max=14,
    description='Proxies',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(proxy_w)
proxy_ok = True
try:
    # TMC proxies
    proxy_w.value = 1
    tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
    proxy_w.value = 2
    tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
    proxy_w.value = 3
    tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
    proxy_w.value = 4
    tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

    # CSP.LMC proxies
    proxy_w.value = 5
    csp_control = DeviceProxy("mid-csp/control/0")
    proxy_w.value = 6
    csp_subarray = DeviceProxy("mid-csp/subarray/01")

    # CBF proxies
    proxy_w.value = 6
    cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
    proxy_w.value = 7
    cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

    # Dish Leaf Proxies
    proxy_w.value = 8
    dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")
    proxy_w.value = 9
    dish_leaf_node_ska036 = DeviceProxy("ska_mid/tm_leaf_node/d0036")

    # SDP Proxies
    proxy_w.value = 10
    sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

    # Set devices to adminMode = ONLINE
    csp_control.adminMode = 0
    csp_subarray.adminMode = 0

    # Leaf Nodes
    csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
    sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
    csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"
    proxy_w.value = 11
    csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
    proxy_w.value = 12
    sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
    proxy_w.value = 13
    csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)
    proxy_w.value = 14
except tango.DevFailed as terr:
    # print(f"ERROR: {terr.args[0].desc.strip()}")
    show_errorbox(terr.args[0].desc.strip())
    tmc_central_node = None
    tmc_csp_master = None
    proxy_ok = False
    
proxy_ok_w = widgets.Valid(
    value=proxy_ok,
    description="Proxies OK",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(proxy_ok_w)

sleep(2)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = CBF_SIMULATION_MODE
csp_control.commandTimeout = 99

sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

Setting up the device proxies


IntProgress(value=0, description='Proxies', max=14, style=ProgressStyle(bar_color='maroon'))

Valid(value=True, description='Proxies OK', layout=Layout(width='60%'), style=DescriptionStyle(description_wid…


Checking admin mode after setting to ONLINE (0):
  CSP Control: 0
  CSP Subarray: 0
  CBF Controller: 0
  CBF Subarray: 0

Checking CBF Simulation Mode and CBF Timeout:
  CBF Simulation Mode: True
  CBF Timeout: 99 sec


## Load the Dish Vcc Config / Init Sys Params

In [47]:
import tango

for n in range(1, 3):
    print(f"Load config {n}\n")
    with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
        dish_config_json = json.load(f)

    dish_config_json["tm_data_sources"][
        0
    ] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
    dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

    print(f'dish_config_json tm_data_filepath: \n\t{dish_config_json["tm_data_filepath"]}')
    try:
        tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
    except tango.DevFailed as terr:
        show_errorbox(terr.args[0].desc.strip(), "Source dish error")
    except Exception as tmc_err:
        print(f"ERROR: {str(tmc_err)}")

    sleep(2)
    try:
        dish_vcc_cfg = json.loads(tmc_csp_master.sourceDishVccConfig)
        print('\nTMC CSP Master Source Dish Vcc Config dish parameters:')
        for key in dish_vcc_cfg:
            print(f"\t{key:20} : {dish_vcc_cfg[key]}")
    except tango.DevFailed as terr:
        show_errorbox(terr.args[0].desc.strip(), "Source dish error")
    print()

Load config 1

dish_config_json tm_data_filepath: 
	instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json


Textarea(value='TRANSIENT CORBA system exception: TRANSIENT_CallTimedout', description='Source dish error', la…


TMC CSP Master Source Dish Vcc Config dish parameters:
	interface            : https://schema.skao.int/ska-mid-cbf-initsysparam/1.0
	tm_data_sources      : ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata']
	tm_data_filepath     : instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json

Load config 2

dish_config_json tm_data_filepath: 
	instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json


Textarea(value='TRANSIENT CORBA system exception: TRANSIENT_CallTimedout', description='Source dish error', la…


TMC CSP Master Source Dish Vcc Config dish parameters:
	interface            : https://schema.skao.int/ska-mid-cbf-initsysparam/1.0
	tm_data_sources      : ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata']
	tm_data_filepath     : instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json



## Smoke test

In [11]:
# Smoke test
def smoke_test(tango_dev, tango_desc):
    smoke = False
    if tango_dev is None:
        smoke = False
    else:
        try:
            tping = tango_dev.ping()
            # print(f"Ping TMC central node: {tping}")
            smoke = True
        except tango.ConnectionFailed as terr:
            err_msg = terr.args[0].desc.strip()
            # print(f"Tango connection failed: {err_msg}")
            show_errorbox(err_msg)
            smoke = False
    smoke_w = widgets.Valid(
        value=smoke,
        description=f"{tango_desc} online",
        layout=widgets.Layout(width='60%'),
        style = {'description_width': 'initial'}
    )
    display(smoke_w)

smoke_test(tmc_central_node, "TMC Central Node")
smoke_test(tmc_csp_master, "TMC CSP master")
smoke_test(tmc_csp_subarray, "TMC CSP subarray"),
smoke_test(tmc_subarray, "TMC subarray"),
smoke_test(csp_control, "CSP control"),
smoke_test(csp_subarray, "CSP subarray"),
smoke_test(cbf_controller, "CBF controller"),
smoke_test(cbf_subarray, "CBF subarray")
smoke_test(dish_leaf_node_ska001, "Dish leaf 001")
smoke_test(dish_leaf_node_ska036, "Dish leaf 036")
smoke_test(sdp_subarray, "SDP subarray")

Valid(value=True, description='TMC Central Node online', layout=Layout(width='60%'), style=DescriptionStyle(de…

Valid(value=True, description='TMC CSP master online', layout=Layout(width='60%'), style=DescriptionStyle(desc…

Valid(value=True, description='TMC CSP subarray online', layout=Layout(width='60%'), style=DescriptionStyle(de…

Valid(value=True, description='TMC subarray online', layout=Layout(width='60%'), style=DescriptionStyle(descri…

Valid(value=True, description='CSP control online', layout=Layout(width='60%'), style=DescriptionStyle(descrip…

...

Valid(value=True, description='CBF controller online', layout=Layout(width='60%'), style=DescriptionStyle(desc…

Valid(value=True, description='CBF subarray online', layout=Layout(width='60%'), style=DescriptionStyle(descri…

Valid(value=True, description='Dish leaf 001 online', layout=Layout(width='60%'), style=DescriptionStyle(descr…

Valid(value=True, description='Dish leaf 036 online', layout=Layout(width='60%'), style=DescriptionStyle(descr…

Valid(value=True, description='SDP subarray online', layout=Layout(width='60%'), style=DescriptionStyle(descri…

## Admin mode check

In [12]:
def admin_test(tango_dev, tango_desc):
    dev_admin = tango_dev.adminMode
    admin_w = widgets.Checkbox(
        value=not dev_admin,
        description=f"{tango_desc} admin",
        disabled=False,
        indent=False
    )
    display(admin_w)
    
admin_test(tmc_central_node, "TMC Central Node")
admin_test(tmc_csp_master, "TMC CSP master")
admin_test(tmc_csp_subarray, "TMC CSP subarray"),
admin_test(tmc_subarray, "TMC subarray"),
admin_test(csp_control, "CSP control"),
admin_test(csp_subarray, "CSP subarray"),
admin_test(cbf_controller, "CBF controller"),
admin_test(cbf_subarray, "CBF subarray")
admin_test(dish_leaf_node_ska001, "Dish leaf 001")
admin_test(dish_leaf_node_ska036, "Dish leaf 036")
admin_test(sdp_subarray, "SDP subarray")


Checkbox(value=False, description='TMC Central Node admin', indent=False)

Checkbox(value=False, description='TMC CSP master admin', indent=False)

Checkbox(value=False, description='TMC CSP subarray admin', indent=False)

Checkbox(value=False, description='TMC subarray admin', indent=False)

Checkbox(value=True, description='CSP control admin', indent=False)

...

Checkbox(value=True, description='CBF controller admin', indent=False)

Checkbox(value=True, description='CBF subarray admin', indent=False)

Checkbox(value=False, description='Dish leaf 001 admin', indent=False)

Checkbox(value=False, description='Dish leaf 036 admin', indent=False)

Checkbox(value=True, description='SDP subarray admin', indent=False)

# Set up the scan

## Turn the Telescope On
Known issues:
When turning on Telescope, talon DDR memory calibration can fail. An indicator of this is mid_csp_cbf/fspcoorrsubarray obsState = FAULT. To adress this:
* Manually power off of the PDUs to the talon boards
* Rerun of the Telescope ON command.

### Run the Telescope On command

In [13]:
import tango

# TODO: Check states
print("Set up device proxy for CBF FSP correlator subarray")
cbf_fspcorrsubarray = DeviceProxy("mid_csp_cbf/fspcorrsubarray/01_01")
print(f"CBF FSP correlator subarray observation {str(cbf_fspcorrsubarray.obsstate)}")
assert cbf_fspcorrsubarray.obsstate == 2

print("Running the TelescopeOn command")
try:
    tmc_central_node.TelescopeOn()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    on_error_w = widgets.Textarea(
        value=err_msg,
        placeholder='Error message',
        description='Ignore',
        disabled=False,
        layout=widgets.Layout(width='75%', height='200px')
    )
    display(on_error_w)

Set up device proxy for CBF FSP correlator subarray
CBF FSP correlator subarray observation obsState.IDLE
Running the TelescopeOn command


### Wait for Telescope to come ON

In [14]:
try:
    print(f"Telescope state {str(tmc_central_node.telescopeState)}")
except tango.DevFailed as terr:
    show_errorbox(terr.args[0].desc.strip(), "State")

wait_time = tel_on_wait.value

# Check if telescope is on
sleep_time = 5
nmax = int(wait_time/sleep_time)

prog_w = widgets.IntProgress(
    value=0,
    min=0,
    max=nmax,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(prog_w)
n = 0
tel_state = -1
errorbox_on = widgets.Textarea(
    value="",
    placeholder='Error message',
    description='State error',
    disabled=False,
    layout=widgets.Layout(width='75%', height='200px')
)

# while tel_state != 0:
while n < nmax:
    n += 1
    prog_w.value = n
    try:
        tel_state = int(tmc_central_node.telescopeState)
    except tango.DevFailed as t_err:
        err_msg: str = t_err.args[0].desc.strip()
        # print(f"Could not read telescope state: {err_msg}")
        errorbox_on.value = err_msg
        display(errorbox_on)
    if tel_state == 0:
        break
    try:
        sleep(sleep_time)
    except KeyboardInterrupt:
        print("Abort")
        break
    
tel_on = widgets.Valid(
    value=not tmc_central_node.telescopeState,
    description="Telescope ON",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(tel_on)

Telescope state tango._tango.DevState(0)


IntProgress(value=0, description='Loading:', max=36, style=ProgressStyle(bar_color='maroon'))

Valid(value=True, description='Telescope ON', layout=Layout(width='60%'), style=DescriptionStyle(description_w…

### Verify the states

In [15]:
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

Verifying the states:
  TMC Central Node State: ON
  CSP Control State: ON
  CBF Controller State: ON
  TMC Subarray State: ON


## Assign Resources
//Vis-receive pods comes up in the staging-sdp namespace 
Known issues:
1. The mid_csp_cbf/sub_elt/subarray_01 device times out. To resolve:
* Rerun the assign resources block

2. eb and pb ids already exist (logs in ds-subarraynode-01-0 pod). Resolution:
* Edit eb and pb ids in the assign_resources.json


In [16]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

print(f"sdp subarray state {sdp_subarray.state()}")

sleep(3)

print(
    "Running the AssignResources command:"
    " subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS
    assign_resources_json["sdp"]["processing_blocks"][0]["parameters"][
        "queue_connector_configuration"
    ]["exchanges"][0]["source"]["servers"] = KAFKA_ENDPOINT
    assign_resources_json["sdp"]["processing_blocks"][0]["parameters"]["extra_helm_values"][
        "receiver"
    ]["options"]["reception"][
        "stats_receiver_kafka_config"
    ] = f"{KAFKA_ENDPOINT}:json_workflow_state"

# print(f"\nassign_resources_json file contents: \n{assign_resources_json}")
RenderJSON(assign_resources_json)

try:
    tmc_subarray.AssignResources(json.dumps(assign_resources_json))
except tango.DevFailed as terr:
    show_errorbox(terr.args[0].desc.strip())

sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

sdp subarray state ON
Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned

CBF Subarray Observation State: 1
CBF Subarray Receptors : ('SKA001', 'SKA036')


# Start scanning

## Slew the dishes to near the start of track to avoid tracking timing errors

Could take +1min

In [19]:
import time

SKA001_TANGO_HOST = f"tango-databaseds.{SKA001_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
dish_manager_ska001 = DeviceProxy(f"tango://{SKA001_TANGO_HOST}/mid-dish/dish-manager/ska001")
SKA036_TANGO_HOST = f"tango-databaseds.{SKA036_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
dish_manager_ska036 = DeviceProxy(f"tango://{SKA036_TANGO_HOST}/mid-dish/dish-manager/ska036")

dish_manager_ska001.slew([181.0, 31.0])
dish_manager_ska036.slew([181.0, 31.0])

slew001_w = widgets.IntProgress(
    value=1,
    min=0,
    max=wait_time,
    description='Slew 001:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(slew001_w)

slew001_ok = True
n = 0
while (
    dish_manager_ska001.achievedpointing[1] != 181.0
    or dish_manager_ska001.achievedpointing[2] != 31.0
):
    n += 1
    slew001_w.value = n
    if n >= wait_time:
        slew001_ok = False
        break
    try:
        time.sleep(1)
    except KeyboardInterrupt:
        slew001_ok = False
        break
        
    
slew001ok_w = widgets.Valid(
    value=slew001_ok,
    description=f"Slew 001",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(slew001ok_w)

slew036_w = widgets.IntProgress(
    value=1,
    min=0,
    max=wait_time,
    description='Slew 036:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(slew036_w)

slew036_ok = True
n = 0
while (
    dish_manager_ska036.achievedpointing[1] != 181.0
    or dish_manager_ska036.achievedpointing[2] != 31.0
):
    n += 1
    slew036_w.value = n
    if n >= wait_time:
        slew036_ok = False
        break
    try:
        time.sleep(1)
    except KeyboardInterrupt:
        slew001_ok = False
        break
    
slew036ok_w = widgets.Valid(
    value=slew036_ok,
    description=f"Slew 001",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(slew036ok_w)

# print("Done slewing")

IntProgress(value=1, description='Slew 001:', max=180, style=ProgressStyle(bar_color='maroon'))

Valid(value=True, description='Slew 001', layout=Layout(width='60%'), style=DescriptionStyle(description_width…

IntProgress(value=1, description='Slew 036:', max=180, style=ProgressStyle(bar_color='maroon'))

Valid(value=True, description='Slew 001', layout=Layout(width='60%'), style=DescriptionStyle(description_width…

## Configure Scan

In [20]:
# TODO: Check here whether the states are as expected first
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

# print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

RenderJSON(configure_scan_json)
# print(json.dumps(configure_scan_json))

try:
    tmc_subarray.Configure(json.dumps(configure_scan_json))
except tango.DevFailed as terr:
    show_errorbox(terr.args[0].desc.strip())
    
cfg_w = widgets.IntProgress(
    value=1,
    min=0,
    max=12,
    description='Configure',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(cfg_w)

for n in range(1, 13):
    cfg_w.value = n
    sleep(1)

show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP Subarray Observation State")
show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP Subarray Observation State")
# print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
# print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")


Running the Configure command: subarray obsstate should go to Ready


IntProgress(value=1, description='Configure scan:', max=12, style=ProgressStyle(bar_color='maroon'))

Text(value='sdpSubarrayObsState.READY', description='SDP Subarray Observation State', disabled=True, layout=La…

Text(value='cspSubarrayObsState.IDLE', description='CSP Subarray Observation State', disabled=True, layout=Lay…

## Check subarray observation states

In [21]:
# Wait about 5 seconds

sub_chk_w = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Check obs:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(sub_chk_w)

for n in range(1, 11):
    sub_chk_w.value = n
    sleep(1)

# print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")
# print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
# print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

cbf_chk = (
    (cbf_subarray.obsState == 4)
    and (sdp_subarray_leaf_node_dp.sdpSubarrayObsState == 4)
    and (csp_subarray_leaf_node_dp.cspSubarrayObsState == 4)
)

cbf_obs_state = cbf_subarray.obsState
cbf_chk_w1 = widgets.Valid(
    value=(cbf_obs_state == 4),
    description=f"CBF Subarray Observation State ({cbf_obs_state})",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_chk_w1)

sdp_obs_state = sdp_subarray_leaf_node_dp.sdpSubarrayObsState
cbf_chk_w2 = widgets.Valid(
    value=(sdp_obs_state == 4),
    description=f"SDP Subarray Observation State ({sdp_obs_state})",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_chk_w2)

csp_obs_state = csp_subarray_leaf_node_dp.cspSubarrayObsState
cbf_chk_w3 = widgets.Valid(
    value=(csp_obs_state == 4),
    description=f"CSP Subarray Observation State ({csp_obs_state})",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_chk_w3)

IntProgress(value=0, description='Check obs:', max=10, style=ProgressStyle(bar_color='maroon'))

Valid(value=True, description='CBF Subarray Observation State (4)', layout=Layout(width='60%'), style=Descript…

Valid(value=True, description='SDP Subarray Observation State (4)', layout=Layout(width='60%'), style=Descript…

Valid(value=True, description='CSP Subarray Observation State (4)', layout=Layout(width='60%'), style=Descript…

## Scan

In [22]:
if not APPLY_HACKS:
    # DO NOT USE FOR NOW
    print("Running the Scan command: subarray obsstate should go to Scanning")

    # with open(SCAN_FILE, encoding="utf-8") as f:
    #     scan_json = f.read()

    # print(f"\nscan_json file contents: \n{scan_json}")

    # tmc_subarray.Scan(scan_json)

    # sleep(2)
    # print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")
else:
    # TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
    with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
        d = json.load(json_data)
        sdp_scan = d["sdp"]
        print(sdp_scan)
        try:
            sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))
        except tango.DevFailed as terr:
            show_errorbox(terr.args[0].desc.strip())
        
    leaf_node_w = widgets.IntProgress(
        value=0,
        min=0,
        max=10,
        description='Subarrays',
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        style={'bar_color': 'maroon'},
        orientation='horizontal'
    )
    display(leaf_node_w)

    for n in range(1, 11):
        leaf_node_w.value = n
        sleep(1)
    print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

    with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
        d = json.load(json_data)
        csp_scan = d["csp"]
        print(f"CSP scan:\n{csp_scan}")
        try:
            csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))
        except tango.DevFailed as terr:
            show_errorbox(terr.args[0].desc.strip())

    sleep(2)
    print(f"\ncsp Subarray Obs State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

{'interface': 'https://schema.skao.int/ska-sdp-scan/0.4', 'scan_id': 1}


IntProgress(value=0, description='Subarrays', max=10, style=ProgressStyle(bar_color='maroon'))

sdpSubarrayObsState.SCANNING
CSP scan:
{'interface': 'https://schema.skao.int/ska-csp-scan/2.2', 'scan_id': 1}

csp Subarray Obs State: 5


# Health checks

## Print TMC diagnostics

In [23]:
# Print TMC diagnostics
tmc_table: str = '<style>td {border: thin blue ridge; padding: 2px}</style><table>'
try:
    tmc_table += f"<tr><td>Central Node</td><td>{tmc_central_node.name()}</td></tr>"
    tmc_table += f"<tr><td>Central Node state</td><td>{str(tmc_central_node.State())}</td></tr>"
    tmc_table += f"<tr><td>Central Node adminMode</td><td>{str(tmc_central_node.adminmode)}</td></tr>"
    tmc_table += f"<tr><td>Central Node healthState</td><td>{str(tmc_central_node.healthState)}</td></tr>"
    tmc_table += \
        f"<tr><td>Central Node telescopeHealthState</td>" \
        f"<td>{str(tmc_central_node.telescopeHealthState)}</td></tr>"
    tmc_table += \
        f"<tr><td>Central Node isDishVccConfig</td>" \
        f"<td>{str(tmc_central_node.isDishVccConfigSet)}</td></tr>"
    tmc_table += \
        f"<tr><td>Central Node DishVccValidationStatus</td>" \
        f"<td>{str(tmc_central_node.DishVccValidationStatus)}</td></tr>"
    tmc_table += f"<tr><td>Subarray Node state</td><td>{tmc_csp_subarray.State()}</td></tr>"
    tmc_table += f"<tr><td>Subarray adminMode</td><td>{str(tmc_csp_subarray.adminMode)}</td></tr>"
    # print(f"TMC Subarray Node obsState: {str(tmc_csp_subarray.obsState)}")
except tango.ConnectionFailed as terr:
    print(f"Tango connection failed: {terr.args[0].desc.strip()}")
# table(tmc_rows, frame=True)
tmc_table += "</table>"
tmc_table_w = widgets.HTML(
    value=tmc_table,
    placeholder='TMC status',
    description='TMC',
)
display(tmc_table_w)

HTML(value='<style>td {border: thin blue ridge; padding: 2px}</style><table><tr><td>Central Node</td><td>ska_m…

## Print CSP-LMC Diagnostics

In [24]:
# Print CSP-LMC Diagnostics
csplmc_table: str = "<table>"
try:
    # Controller
    csplmc_table += f"<tr><td>Controller</td><td>{csp_control.name()}</td></tr>"
    csplmc_table += f"<tr><td> Controller admin mode</td><td>{str(csp_control.adminMode)}</td></tr>"
    csplmc_table += f"<tr><td>Controller State</td><td>{csp_control.State()}</td></tr>"
    csplmc_table += f"<tr><td>Controller dish Vcc Config</td><td>{csp_control.dishVccConfig}</td></tr>"
    csplmc_table += \
        f"<tr><td>Controller CBF Simulation Mode</td>" \
        f"<td>{csp_control.cbfSimulationMode:}</td></tr>"
    # Subarray
    csplmc_table += f"<tr><td>Subarray</td><td>{csp_subarray.name()}</td></tr>"
    csplmc_table += f"<tr><td>Subarray admin mode</td><td>{str(csp_subarray.adminMode)}</td></tr>"
    csplmc_table += f"<tr><td>Subarray State</td><td>{csp_subarray.State()}</td></tr>"
    csplmc_table += f"<tr><td>Subarray observation state</td><td>{str(csp_subarray.obsState)}</td></tr>"
    csplmc_table += f"<tr><td>Subarray dish Vcc Config</td><td>{csp_subarray.dishVccConfig}</td></tr>"
    csplmc_table += f"<tr><td>FSP correlator subarray</td><td>{cbf_fspcorrsubarray.obsstate}</td></tr>"
except tango.ConnectionFailed as terr:
    # print(f"Tango connection failed: {terr.args[0].desc.strip()}")
    show_errorbox(terr.args[0].desc.strip())
csplmc_table += "</table>"
csplmc_table_w = widgets.HTML(
    value=csplmc_table,
    placeholder='CSP-LMC status',
    description='CSP-LMC',
)
display(csplmc_table_w)

HTML(value='<table><tr><td>Controller</td><td>mid-csp/control/0</td></tr><tr><td> Controller admin mode</td><t…

## Print CBF Diagnostics

In [25]:
# Print CBF Diagnostics
cbf_table: str = "<table>"
try:
    # CBF Controller
    cbf_table += f"<tr><td>Controller admin mode</td><td>{str(cbf_controller.adminMode)}</td></tr>"
    cbf_table += f"<tr><td>Controller State</td><td>{str(cbf_controller.State())}</td></tr>"
    # CBF Subarray
    cbf_table += f"<tr><td>Subarray admin mode</td><td>{str(cbf_subarray.adminMode)}</td></tr>"
    cbf_table += f"<tr><td>Subarray State</td><td>{str(cbf_subarray.State())}</td></tr>"
    cbf_table += f"<tr><td>Subarray observation state</td><td>{str(cbf_subarray.obsState)}</td></tr>"
except tango.ConnectionFailed as terr:
    err_msg = terr.args[0].desc.strip()
    # print("Tango connection failed: %s", err_msg)
    show_errorbox(err_msg)
cbf_table += "</table>"
cbf_table_w = widgets.HTML(
    value=cbf_table,
    placeholder='CBF status',
    description='CBF',
)
display(csplmc_table_w)

HTML(value='<table><tr><td>Controller</td><td>mid-csp/control/0</td></tr><tr><td> Controller admin mode</td><t…

## Print SDP Diagnostics

In [26]:
# Print SDP Diagnostics
sdp_table: str = "<table>"
try:
    # SDP Subarray
    sdp_table += f"<tr><td>Subarray state</td><td>{str(sdp_subarray.State())}</td></tr>"
    sdp_table += f"<tr><td>Subarray admin mode</td><td>{str(sdp_subarray.adminMode)}</td></tr>"
    sdp_table += f"<tr><td>Subarray observation state</td><td>{str(sdp_subarray.obsState)}</td></tr>"
except tango.ConnectionFailed as terr:
    err_msg = terr.args[0].desc.strip()
    # print("Tango connection failed: %s", err_msg)
    show_errorbox(err_msg)
sdp_table += "</table>"
sdp_table_w = widgets.HTML(
    value=cbf_table,
    placeholder='SDP status',
    description='SDP',
)
display(sdp_table_w)

HTML(value='<table><tr><td>Controller admin mode</td><td>adminMode.ONLINE</td></tr><tr><td>Controller State</t…

## Check subarrays

In [27]:
# Wait about 5 seconds and show subarrays
cbf_prog_w = widgets.IntProgress(
    value=0,
    min=0,
    max=5,
    description='Subarrays',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(cbf_prog_w)

for n in range(1,6):
    cbf_prog_w.value = n
    sleep(1)

show_textbox(str(cbf_subarray.obsState), "CBF Subarray Observation")
show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP Subarray Observation")
show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP Subarray Observation")
cbf_sub_valid = (
    (cbf_subarray.obsState == 4) 
    and (sdp_subarray_leaf_node_dp.sdpSubarrayObsState == 4) 
    and (csp_subarray_leaf_node_dp.cspSubarrayObsState == 4)
)
cbf_sub_w = widgets.Valid(
    value=cbf_sub_valid,
    description="CSP Subarray",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_sub_w)

IntProgress(value=0, description='Subarrays', max=5, style=ProgressStyle(bar_color='maroon'))

Text(value='obsState.SCANNING', description='CBF Subarray Observation', disabled=True, layout=Layout(width='60…

Text(value='sdpSubarrayObsState.SCANNING', description='SDP Subarray Observation', disabled=True, layout=Layou…

Text(value='cspSubarrayObsState.SCANNING', description='CSP Subarray Observation', disabled=True, layout=Layou…

Valid(value=False, description='CSP Subarray', layout=Layout(width='60%'), style=DescriptionStyle(description_…

# Shut down

## End scan

In [30]:
if not APPLY_HACKS:
    # DO NOT USE FOR NOW SINCE TMC SUBARRAY IS STUCK IN CONFIGURING
    print("Run the End Scan command: subarray obsstate should go to Ready state")

    # tmc_subarray.EndScan()

    # sleep(2)
    # print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")
else:
    # TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
    print("Running the End Scan command: subarray obsstate should go to Ready state")

    try:
        sdp_subarray_leaf_node_dp.EndScan()
    except tango.DevFailed as terr:
        show_errorbox(terr.args[0].desc.strip(), "SDP end scan")
    sleep(2)
    try:
        csp_subarray_leaf_node_dp.EndScan()
    except tango.DevFailed as terr:
        show_errorbox(terr.args[0].desc.strip(), "CSP end scan")
    show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP state")
    show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP state")

Running the End Scan command: subarray obsstate should go to Ready state


Textarea(value='ska_tmc_common.exceptions.InvalidObsStateError: EndScan command is not allowed in current obse…

Textarea(value='ska_tmc_common.exceptions.InvalidObsStateError: EndScan command is not allowed in current obse…

Text(value='sdpSubarrayObsState.READY', description='SDP state', disabled=True, layout=Layout(width='60%'))

Text(value='cspSubarrayObsState.READY', description='CSP state', disabled=True, layout=Layout(width='60%'))

## End subarrays

In [33]:
if not APPLY_HACKS:
    # DO NOT USE (TMC STUCK IN CONFIGURING)
    print("Run the End command: subarray obsstate should go to Idle state")

    # tmc_subarray.End()

    # sleep(2)
    # print(f"\TMC Subarray Observation State: {cbf_subarray.obsState}")
else:
    # TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
    print("Running the End command: subarray obsstate should go to Idle state")

    sdp_subarray_leaf_node_dp.End()
    sleep(2)
    csp_subarray_leaf_node_dp.End()

    sleep(2)
    show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP state")
    show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP state")

Running the End command: subarray obsstate should go to Idle state


Text(value='sdpSubarrayObsState.IDLE', description='SDP state', disabled=True, layout=Layout(width='60%'))

Text(value='cspSubarrayObsState.IDLE', description='CSP state', disabled=True, layout=Layout(width='60%'))

## Release Resources

In [34]:
if not APPLY_HACKS:
    # DO NOT USE (TMC STUCK IN CONFIGURING)
    print(
        "Run the Release All Resources command:"
        " subarray obsstate should go to Empty state and receptor IDs should be empty"
    )

    # tmc_subarray.ReleaseAllResources()

    # sleep(2)
    # print(f"TMC Subarray Observation State: {tmc_subarray.obsState}")
else:
    # TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
    print(
        "Running the Release All Resources command:"
        " subarray obsstate should go to Empty state and receptor IDs should be empty"
    )

    sdp_subarray_leaf_node_dp.ReleaseAllResources()
    sleep(2)
    csp_subarray_leaf_node_dp.ReleaseAllResources()

    sleep(2)
    show_textbox(str(sdp_subarray_leaf_node_dp.sdpSubarrayObsState), "SDP state")
    show_textbox(str(csp_subarray_leaf_node_dp.cspSubarrayObsState), "CSP state")

Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty


Text(value='sdpSubarrayObsState.EMPTY', description='SDP state', disabled=True, layout=Layout(width='60%'))

Text(value='cspSubarrayObsState.EMPTY', description='CSP state', disabled=True, layout=Layout(width='60%'))

SDP Subarray Observation State: 0
CSP Subarray Observation State: 0


## Telescope Off

In [36]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

off_prog_w = widgets.IntProgress(
    value=0,
    min=0,
    max=5,
    description='Turning off',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(off_prog_w)

for n in range(1,6):
    off_prog_w.value = n
    sleep(1)

show_textbox(str(tmc_central_node.State()), "TMC central")
show_textbox(str(csp_control.State()), "CSP control")
show_textbox(str(cbf_controller.State()), "CBF control")

Running the TelescopeOff command


IntProgress(value=0, description='Turning off', max=5, style=ProgressStyle(bar_color='maroon'))

Text(value='ON', description='TMC central', disabled=True, layout=Layout(width='60%'))

Text(value='ON', description='CSP control', disabled=True, layout=Layout(width='60%'))

Text(value='ON', description='CBF control', disabled=True, layout=Layout(width='60%'))

# Configure Test Equipment

## Test equipment setup changes as required for some of the mid - aiv jama tests

In [ ]:
from bokeh.io import output_notebook

from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

## Print current status

In [ ]:
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

## Print Test Equipment Diagnostics

In [ ]:
test_equipment.print_diagnostics()

## Create Test Equipment Plot

In [ ]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

Turn offline Test Equipment devices ONLINE

In [ ]:
# set any offline devices to online
test_equipment.turn_online()

## Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

Band 1 setup

Set CW generator to approximately 480MHz (freq1) & -30dBm (power1), and wideband noise to approximately -132dBm/Hz at the SPF Band 1 Receiver inputs. ie. Band 1 output on and attenuation approx 10dB.  

Expected Result:  
*Test equipment configured*

In [ ]:
# test equipment setup values - band 1; use 950.000e6 for frequency for band 2

import time

frequency_to_set = 480.000e6
power_level = -30.0
attenuation = 10
band = 1

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(3)
# print updated values and confirm updates
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == power_level, print(
    f"Power level required is {power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)

## print current setup

In [ ]:
# print current only
signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

~10 seconds after recording has started, use Test Control Script to adjust CW generator signal level by -3dB. 

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [ ]:
time.sleep(
    10
)  # cell to be run in this notebook immediately after starting scn in rf-chain-and-channelisation_test_equipment notebook
power_level_minus_3_db = power_level - 3.0
signal_generator = test_equipment.signal_generator
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute("power_dbm", power_level_minus_3_db)
time.sleep(3)
print(f"Updated signal generator power level: {signal_generator.power_dbm}")

After a further ~10 seconds, use Test Control Script to adjust CW generator signal level by -3dB.  

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [ ]:
time.sleep(
    10
)  # cell to be run in this notebook immediately after starting scn in rf-chain-and-channelisation_test_equipment notebook
power_level_minus_6_db = power_level - 6.0
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute("power_dbm", power_level_minus_6_db)
time.sleep(3)
print(f"Updated signal generator power level: {signal_generator.power_dbm}")

After a further ~10seconds, use Test Control Script to increase the CW generator signal frequency by 10MHz.  

Expected Result:  
*CW frequency changes by 10MHz in SDP Spectrometer Display*

In [ ]:
time.sleep(10)
frequency_plus_10MHz = frequency_to_set + 10.0e6
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute("frequency", frequency_plus_10MHz)
time.sleep(3)
print(f"Updated signal generator frequency: {signal_generator.frequency}")